In [ ]:
import requests

def read_dataset_from_url(url):
    response = requests.get(url)

    # Ensure request was successful
    if response.status_code != 200:
        raise Exception(f"Failed to download file: {response.status_code}")

    lines = response.text.strip().split("\n")  # Split text by lines

    # Read first line: total flights (N) and total pairings (M)
    flight_count, pairing_count = map(int, lines[0].split())

    # Read crew pairings
    pairings = []
    for line in lines[1:]:  # Skip first line
        data = list(map(int, line.split()))
        cost = data[0]   # First number is cost
        flights = data[2:]  # Flights covered (skip second number)
        pairings.append([cost, len(flights)] + flights)  # Store in list format

    return flight_count, pairing_count, pairings

# Example usage
url = "https://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/sppnw41.txt"
flight_count, pairing_count, pairings = read_dataset_from_url(url)

# Print parsed data
print("Total Flights:", flight_count)
print("Total Pairings:", pairing_count)
print("First 5 Pairings:", pairings[:5])  # Display first 5 pairings


Total Flights: 17
Total Pairings: 197
First 5 Pairings: [[2259, 5, 1, 3, 4, 8, 10], [3309, 4, 1, 3, 4, 11], [4497, 3, 1, 3, 4], [4965, 4, 1, 4, 9, 11], [5961, 3, 1, 4, 9]]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Algorithm 2: Pseudo-Random Initialization
def initialize_population(pairings, flight_count, population_size):
    population = np.zeros((population_size, len(pairings)), dtype=int)

    for k in range(population_size):
        Sk = np.zeros(len(pairings), dtype=int)
        U = set(range(1, flight_count + 1))

        while U:
            i = np.random.choice(list(U))
            valid_pairings = [j for j in range(len(pairings)) if i in pairings[j][2:]]
            if valid_pairings:
                j = np.random.choice(valid_pairings)
                Sk[j] = 1
                U -= set(pairings[j][2:])
            else:
                U.remove(i)

        population[k] = Sk

    return population

# Compute cost function
def compute_cost(solution, pairings, flight_count, penalty=1000):
    total_cost = sum(pairings[i][0] for i in range(len(solution)) if solution[i] == 1)
    covered_flights = {flight for i in range(len(solution)) if solution[i] == 1 for flight in pairings[i][2:]}

    # Apply penalty for uncovered flights
    missing_flights = flight_count - len(covered_flights)
    return total_cost + penalty * missing_flights if missing_flights > 0 else total_cost

# Compute constraint violations
def compute_constraint_violations(solution, pairings, flight_count):
    covered_flights = {flight for i in range(len(solution)) if solution[i] == 1 for flight in pairings[i][2:]}
    return max(0, flight_count - len(covered_flights))

# Hybrid Crossover (50% One-point, 50% Uniform)
def hybrid_crossover(parent1, parent2):
    if np.random.rand() < 0.5:
        # One-point crossover
        point = np.random.randint(1, len(parent1))
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
    else:
        # Uniform crossover (swap each gene randomly)
        mask = np.random.rand(len(parent1)) < 0.5
        child1, child2 = parent1.copy(), parent2.copy()
        child1[mask], child2[mask] = child2[mask], child1[mask]

    return child1, child2

# Bit-flip mutation with adaptive rate
def mutation(solution, mutation_rate):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if np.random.rand() < mutation_rate:
            mutated_solution[i] = 1 - mutated_solution[i]
    return mutated_solution

# Tournament selection
def tournament_selection(population, fitness, tournament_size=3):
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = selected_indices[np.argmin(np.array(fitness)[selected_indices])]
    return population[best_index]

# Algorithm 1: Heuristic Improvement Operator
def heuristic_improvement(solution, pairings, flight_count):
    S = set(np.where(solution == 1)[0])
    w = {i: sum(1 for j in S if i in pairings[j][2:]) for i in range(1, flight_count + 1)}
    T = S.copy()

    # DROP procedure
    while T:
        j = np.random.choice(list(T))
        T.remove(j)
        if all(w[i] >= 2 for i in pairings[j][2:]):
            S.remove(j)
            for i in pairings[j][2:]:
                w[i] -= 1

    # ADD procedure
    U = {i for i in range(1, flight_count + 1) if w[i] == 0}
    V = U.copy()

    while V:
        i = np.random.choice(list(V))
        V.remove(i)
        valid_pairings = [j for j in range(len(pairings)) if i in pairings[j][2:]]
        if valid_pairings:
            j = min(valid_pairings, key=lambda j: pairings[j][0] / len(pairings[j][2:]))
            S.add(j)
            for i in pairings[j][2:]:
                w[i] += 1
            U -= set(pairings[j][2:])
            V -= set(pairings[j][2:])

    improved_solution = np.zeros(len(pairings), dtype=int)
    for j in S:
        improved_solution[j] = 1

    return improved_solution

# Improved BGA with Restart and Dynamic Mutation
def bga(pairings, flight_count, population_size=70, generations=100, P_f=0.5):
    population = initialize_population(pairings, flight_count, population_size)
    best_solution, best_cost = None, float('inf')
    # best_cost_per_generation = []
    no_improvement_count = 0
    mutation_rate = 0.2  # Initial mutation rate

    for gen in range(generations):
        # Adaptive mutation increase if stuck
        if no_improvement_count > 5:
            mutation_rate = min(0.6, mutation_rate * 1.5)  # Increase mutation rate

        objective_values = np.array([compute_cost(sol, pairings, flight_count) for sol in population])
        constraint_violations = np.array([compute_constraint_violations(sol, pairings, flight_count) for sol in population])

        min_idx = np.argmin(objective_values)
        if objective_values[min_idx] < best_cost:
            best_solution, best_cost = population[min_idx], objective_values[min_idx]
            no_improvement_count = 0  # Reset counter
            mutation_rate = 0.2  # Reset mutation rate when improvement occurs
        else:
            no_improvement_count += 1

        # best_cost_per_generation.append(best_cost)

        # Restart mechanism if stuck in local minimum
        if no_improvement_count > 15:
            print(f"Restarting some population members at generation {gen}")
            population[:population_size // 2] = initialize_population(pairings, flight_count, population_size // 2)
            no_improvement_count = 0
            mutation_rate = 0.2  # Reset mutation rate after restart

        new_population = []
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, objective_values)
            parent2 = tournament_selection(population, objective_values)

            child1, child2 = hybrid_crossover(parent1, parent2)
            child1 = mutation(child1, mutation_rate)
            child2 = mutation(child2, mutation_rate)

            child1 = heuristic_improvement(child1, pairings, flight_count)
            child2 = heuristic_improvement(child2, pairings, flight_count)

            new_population.extend([child1, child2])

        population = np.array(new_population[:population_size])

    return best_solution, best_cost


In [ ]:
import numpy as np

def run_experiment(pairings, flight_count, num_runs=30):
    costs = []
    all_flights_covered_count = 0  # Track how many times all flights are covered

    for _ in range(num_runs):
        best_solution, best_cost = bga(pairings, flight_count)
        costs.append(best_cost)

        # Check if all flights are covered
        x = set()
        for i in range(len(best_solution)):
            if best_solution[i] == 1:
                print(pairings[i][2:])  # Print selected pairings
                x.update(pairings[i][2:])

        if len(x) == flight_count:
            print("------------------------------")
            print("All flights have been covered")
            all_flights_covered_count += 1

    avg_cost = np.mean(costs)
    std_dev_cost = np.std(costs)

    print(f"\nAverage Best Cost: {avg_cost}")
    print(f"Standard Deviation of Best Cost: {std_dev_cost}")
    print(f"Number of times all flights were covered: {all_flights_covered_count}/{num_runs}")

# Call the function with your data
run_experiment(pairings, flight_count)


Restarting some population members at generation 17
Restarting some population members at generation 40
Restarting some population members at generation 56
Restarting some population members at generation 81
[1, 3, 4, 8, 10]
[2, 7, 11]
[5, 16, 17]
[6, 13, 17]
[9, 14]
[12, 14, 15, 16]
------------------------------
All flights have been covered
Restarting some population members at generation 22
Restarting some population members at generation 38
Restarting some population members at generation 54
Restarting some population members at generation 70
Restarting some population members at generation 98
[1, 3, 4, 8, 10]
[2, 7, 11]
[5]
[6, 12, 13, 16, 17]
[9, 11]
[12, 13, 14, 15]
------------------------------
All flights have been covered
Restarting some population members at generation 27
Restarting some population members at generation 43
Restarting some population members at generation 67
Restarting some population members at generation 83
Restarting some population members at generation